In [1]:
import numpy as np
import os
import yfinance as yf
import pandas as pd
from pathlib import Path
from datetime import datetime,timedelta
from classes import Portfolio
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from asset_database import DICT_ASSET_INFO
from tools import get_value,plot_actual_postion,plot_earnings_in_last_months,get_position
from anbima_token import ANBIMA_TOKEN,credentials
from historico_tesouro.tresury_history import DICT_TRESURY_HISTORY
from transactions import GLOBAL_DATA_BASE

current_path = Path.cwd()

In [2]:
portfolio_luiz = Portfolio(GLOBAL_DATA_BASE[GLOBAL_DATA_BASE["owner"]=="Luiz"].drop("owner",axis=1))
portfolio_luciane = Portfolio(GLOBAL_DATA_BASE[GLOBAL_DATA_BASE["owner"]=="Luciane"].drop("owner",axis=1))

In [3]:
df = portfolio_luiz.data_base[portfolio_luiz.data_base["class"]!="Tesouro"]

In [31]:
dividends_data_base = pd.DataFrame(columns=["date","ticker","dividend","amount","total_dividend"])

for ticker in df["ticker"].unique():

    dividend_data = yf.Ticker(DICT_ASSET_INFO[ticker].ticker).dividends
    for dividend_date,dividend in zip(dividend_data.index,dividend_data):
        date = datetime(dividend_date.year,dividend_date.month,dividend_date.day)
        position_in_date = get_position(
            data_base=df,
            ticker=ticker,
            date=date,
        )[1]
        if position_in_date > 0:

            dict_dividend = {
                "date":dividend_date,
                "ticker":ticker,
                "dividend":dividend,
                "amount":position_in_date,
                "total_dividend":position_in_date * dividend,
            }

            if DICT_ASSET_INFO[ticker].asset_class == "USD":

                earlier = date - timedelta(days=7)
                usd_brl = yf.Ticker("BRL%3DX").history(start=earlier,end=date)[["Close"]]
                usd_brl = usd_brl.sort_index(ascending=False).iloc[0].values[0]
                dict_dividend["dividend"] = dict_dividend["dividend"] * usd_brl
                dict_dividend["total_dividend"] = dict_dividend["total_dividend"] * usd_brl

            dividends_data_base.loc[len(dividends_data_base)] = dict_dividend

dividends_data_base = dividends_data_base.sort_values(by="date",ignore_index=True)

In [ ]:
import tkinter as tk

def on_click():
    portfolio_luciane.plot_actual_postion()

app = tk.Tk()
app.title("Simple Tkinter App")

label = tk.Label(app, text="Hello, Tkinter!")
label.pack()

button = tk.Button(app, text="Click Me", command=on_click)
button.pack()

app.mainloop()